<a href="https://colab.research.google.com/github/PrathamBabaria/color-magnitude-diagram/blob/main/Color_Magnitude_Diagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl

In [2]:
from google.colab import files

In [3]:
mpl.rcParams['font.family']='serif'
mpl.rcParams['font.size']=14

In [4]:
def InitializePlot():
    plt.clf()
    f= plt.figure(figsize=(8,8))
    plt.xlim(-1, 9)
    plt.ylim(24, 18)
    plt.xlabel('F475W - F814W (mag)')
    plt.ylabel('F814W (mag)')

In [5]:
def WriteCatalog(catalogName, params, paramNameString):
    #first, create and open the text file for writing. Choose a name: 
    catalog = open(catalogName, 'w')

    #write first row
    catalog.write('# ' + paramNameString + '\n')

    #then, loop through stars and write 1 row for each star.
    for i in range(len(params[0])): #loop over stars
        for j in range(len(params)): #loop over parameters (ra, dec, v, verr,..)
            catalog.write(str(params[j][i]))
            catalog.write(' ')
        catalog.write('\n')
    catalog.close()
    # the next statement will start a browser download of the file to your 
    # local computer
    files.download(catalogName)

In [6]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
ra, dec, xi, eta, f475w, f814w, v, verr = np.loadtxt('keck_hst_data.txt', unpack = True)
color = f475w - f814w

In [ ]:
f = InitializePlot()
plt.scatter(color, f814w, c = 'gray', s = 3, edgecolors = 'none')

In [ ]:
#Where condition to isolate main sequence stars
ms = (color < 1.75) & (f814w < 23)

#Plot MS stars in blue on the plot and label that region 
f = InitializePlot()
plt.scatter(color, f814w, c = 'gray', s = 3, edgecolors = 'none')
plt.scatter(color[ms], f814w[ms], c = 'blue', edgecolors = 'none', s = 5)
plt.text(-0.5, 23.5, 'MS', color = 'blue', size = 16)

#Write a text file containing information for MS stars by calling the
#WriteCatalog function on the stars that satisfy the MS condition.
#You shouldn't modify the WriteCatalog function itself;
#only copy and modify the line below for the AGB and RGB groups. 
WriteCatalog('MScatalog.txt',
             [ra[ms], dec[ms], xi[ms], eta[ms], f475w[ms], f814w[ms], v[ms], verr[ms]],
             'RA DEC XI ETA F475W F814W V VERR')

#==============================================================
#Replace a, h, and k with numbers, and see what happens 
rgb= (color > 1.75 ) & (f814w > 0.18*(color-2.5)**2.+20.4)
##Add the RGB stars to the plot!
plt.scatter(color[rgb], f814w[rgb], c = 'red', edgecolors = 'none', s = 5)
plt.text(2, 23.5, 'RGB', color = 'magenta', size = 16)
WriteCatalog('RGBcatalog.txt',
             [ra[ms], dec[ms], xi[ms], eta[ms], f475w[ms], f814w[ms], v[ms], verr[ms]],
             'RA DEC XI ETA F475W F814W V VERR')

agb= (color > 1.75) & (f814w < 0.18*(color-2.5)**2.+20.4)
plt.scatter(color[agb], f814w[agb], c = 'black', edgecolors = 'none', s = 5)
plt.text(6, 21, 'AGB', color = 'green', size = 16)
WriteCatalog('AGBcatalog.txt',
             [ra[ms], dec[ms], xi[ms], eta[ms], f475w[ms], f814w[ms], v[ms], verr[ms]],
             'RA DEC XI ETA F475W F814W V VERR')

In [ ]:
nstars = len(color)
print('Fraction of stars on the MS: {0:.3f}'.format(np.sum(ms)/nstars))
print('Fraction of stars on the RGB: {0:.3f}'.format(np.sum(rgb)/nstars))
print('Fraction of stars on the AGB: {0:.3f}'.format(np.sum(agb)/nstars))
selected = ms | rgb | agb
print('Fraction of stars not selected: {0:.3f}'.format(np.sum(~selected)/nstars))